In [11]:
import pandas as pd
from keras.layers import Input, Convolution2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
from keras.callbacks import TensorBoard
data = pd.read_csv('data_before_pca.csv')

In [24]:
data = data.drop('Unnamed: 0',1)
data = data.drop('RECORD',1)

In [25]:
data.head()

,FL/LT/Z5_zscore,FL/BL/Z5_zscore,FL/ST/Z5_zscore,AL/LT/Z5_zscore,AL/BL/Z5_zscore,AL/ST/Z5_zscore,AT/LT/Z5_zscore,AT/BL/Z5_zscore,AT/ST/Z5_zscore,FL/LT/Z3_zscore,...,AT/ST/B_zscore,FL/LT/all_zscore,FL/BLD/all_zscore,FL/STO/all_zscore,AL/LT/all_zscore,AL/BLD/all_zscore,AL/STO/all_zscore,AT/LT/all_zscore,AT/BLD/all_zscore,AT/STO/all_zscore
0,-0.220546,0.830441,0.553744,-0.062170,0.313481,0.109440,-0.087380,0.431826,0.304822,-0.091404,...,0.264531,-0.300438,0.280485,0.165647,-0.269862,0.209701,0.002079,-0.292087,0.273926,-0.292087
1,5.163675,8.494650,5.812008,0.738546,1.175860,0.471037,2.276598,4.389930,3.180664,1.716834,...,2.568802,2.305804,3.939777,2.891134,0.724328,1.115619,0.417739,2.241707,3.847641,2.241707
2,-0.046199,-0.068684,-0.051182,0.081513,-0.010425,-0.021140,-0.010832,-0.032518,-0.026023,-0.032851,...,-0.000559,-0.216045,-0.148804,-0.147901,-0.091461,-0.130558,-0.148026,-0.210040,-0.145324,-0.210040
3,-0.182030,0.046076,0.130178,-0.019079,0.077144,0.062760,-0.070469,0.026749,0.073166,-0.078468,...,0.078916,-0.281794,-0.094011,-0.053897,-0.216358,-0.038568,-0.051581,-0.273961,-0.091813,-0.273961
4,0.275280,10.588497,29.291116,0.402502,9.110054,15.317611,0.130316,5.471276,16.021824,0.075115,...,12.857790,-0.060432,4.939489,15.060928,0.307087,9.450392,17.484097,-0.058753,4.823973,-0.058753


In [7]:
x_train = data.iloc[:,0:45]
x_train.shape

(1070994, 45)

In [12]:
encoding_dim = 10
input_img = Input(shape=(45,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(45, activation='sigmoid')(encoded)
 
autoencoder = Model(inputs=input_img, outputs=decoded)
encoder = Model(inputs=input_img, outputs=encoded)
 
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
 
decoder = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
 
autoencoder.fit(x_train, x_train, epochs=10, batch_size=256, 
                shuffle=True, validation_data=(x_train, x_train))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1070994 samples, validate on 1070994 samples
Epoch 1/10
1070994/1070994 [==============================] - 18s 16us/step - loss: -0.4326 - val_loss: -0.5615
Epoch 2/10
1070994/1070994 [==============================] - 20s 19us/step - loss: -0.5780 - val_loss: -0.5869
Epoch 3/10
1070994/1070994 [==============================] - 16s 15us/step - loss: -0.5920 - val_loss: -0.5965
Epoch 4/10
1070994/1070994 [==============================] - 17s 16us/step - loss: -0.5986 - val_loss: -0.6004
Epoch 5/10
1070994/1070994 [==============================] - 17s 16us/step - loss: -0.6016 - val_loss: -0.6028
Epoch 6/10
1070994/1070994 [==============================] - 17s 16us/step - loss: -0.6037 - val_loss: -0.6047
Epoch 7/10
1070994/1070994 [==============================] - 17s 16us/step - loss: -0.6052 - val_loss: -0.6060
Epoch 8/10
1070994/1070994 [==============

In [14]:
encoded_imgs = encoder.predict(x_train)
decoded_imgs = decoder.predict(encoded_imgs)

In [28]:
data_after_AE = pd.DataFrame(decoded_imgs,columns=data.columns[0:46])

In [20]:
data_after_AE = data_after_AE.drop('Unnamed: 0',1)

In [34]:
data_after_AE.head()

,FL/LT/Z5_zscore,FL/BL/Z5_zscore,FL/ST/Z5_zscore,AL/LT/Z5_zscore,AL/BL/Z5_zscore,AL/ST/Z5_zscore,AT/LT/Z5_zscore,AT/BL/Z5_zscore,AT/ST/Z5_zscore,FL/LT/Z3_zscore,...,FL/BLD/all_zscore,FL/STO/all_zscore,AL/LT/all_zscore,AL/BLD/all_zscore,AL/STO/all_zscore,AT/LT/all_zscore,AT/BLD/all_zscore,AT/STO/all_zscore,mhtS2,eldS2
0,0.0,0.712970,0.967709,0.000000,0.008320,0.304936,0.0,1.043707e-03,0.030190,0.000000,...,0.929352,0.985994,0.0,4.461873e-01,0.637594,0.000000,0.654945,0.000000,9.842645,1.875302
1,1.0,1.000000,1.000000,0.368821,1.000000,1.000000,1.0,1.000000e+00,1.000000,0.000003,...,1.000000,1.000000,0.0,3.874302e-07,0.000000,0.999975,1.000000,0.999972,NaN,NaN
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,NaN,NaN
3,0.0,0.013213,0.043143,0.000000,0.000004,0.000086,0.0,1.788139e-07,0.000002,0.000000,...,0.008174,0.008451,0.0,3.859252e-03,0.005896,0.000000,0.002648,0.000000,NaN,NaN
4,0.0,1.000000,1.000000,0.000000,1.000000,1.000000,0.0,1.000000e+00,1.000000,0.000000,...,1.000000,1.000000,0.0,1.000000e+00,1.000000,0.000000,1.000000,0.000000,NaN,NaN


In [36]:
data_after_AE = data_after_AE.drop('mhtS2',1)
data_after_AE = data_after_AE.drop('eldS2',1)

In [37]:
diff = data_after_AE-data

In [38]:
diff

,FL/LT/Z5_zscore,FL/BL/Z5_zscore,FL/ST/Z5_zscore,AL/LT/Z5_zscore,AL/BL/Z5_zscore,AL/ST/Z5_zscore,AT/LT/Z5_zscore,AT/BL/Z5_zscore,AT/ST/Z5_zscore,FL/LT/Z3_zscore,...,AT/ST/B_zscore,FL/LT/all_zscore,FL/BLD/all_zscore,FL/STO/all_zscore,AL/LT/all_zscore,AL/BLD/all_zscore,AL/STO/all_zscore,AT/LT/all_zscore,AT/BLD/all_zscore,AT/STO/all_zscore
0,0.220546,-0.117471,0.413964,0.062170,-0.305161,0.195495,0.087380,-0.430782,-0.274632,0.091404,...,-0.264413,0.300438,0.648866,0.820346,0.269862,0.236487,0.635515,0.292087,0.381020,0.292087
1,-4.163675,-7.494650,-4.812008,-0.369725,-0.175860,0.528963,-1.276598,-3.389930,-2.180664,-1.716831,...,-1.568802,-1.305804,-2.939777,-1.891134,-0.724328,-1.115619,-0.417739,-1.241731,-2.847641,-1.241735
2,0.046199,0.068684,0.051182,-0.081513,0.010425,0.021140,0.010832,0.032518,0.026023,0.032851,...,0.000559,0.216045,0.148804,0.147901,0.091461,0.130558,0.148026,0.210040,0.145324,0.210040
3,0.182030,-0.032864,-0.087035,0.019079,-0.077140,-0.062674,0.070469,-0.026749,-0.073164,0.078468,...,-0.078916,0.281794,0.102185,0.062348,0.216358,0.042428,0.057477,0.273961,0.094461,0.273961
4,-0.275280,-9.588497,-28.291116,-0.402502,-8.110054,-14.317611,-0.130316,-4.471276,-15.021824,-0.075115,...,-11.857790,0.060432,-3.939489,-14.060928,-0.307087,-8.450392,-16.484097,0.058753,-3.823973,0.058753
5,0.030885,0.117917,-1.653029,-0.127067,0.179826,-0.413803,0.004112,0.541610,-0.452960,0.027708,...,-0.185888,0.208633,0.691177,-0.253897,0.033664,0.257343,-0.501759,0.202833,0.700178,0.202833
6,0.152222,-4.451157,-14.192601,-0.016977,-3.775425,-7.047561,0.057382,-1.818149,-7.311085,0.068458,...,-5.679556,0.267366,-1.486656,-6.753324,0.171591,-3.896918,-8.127066,0.259933,-1.428502,0.259933
7,-0.093054,-0.014586,0.416783,-0.127476,-0.118454,0.263973,-0.034441,0.376608,0.386046,-0.011223,...,-0.237307,0.148580,-0.010668,0.002425,-0.057386,-0.111043,-0.045048,0.145068,0.199584,0.145064
8,0.211897,-0.054451,0.089338,0.061712,-0.006531,0.040247,0.083583,-0.031074,0.046892,0.088499,...,0.017280,0.296252,0.090013,0.167678,0.269292,0.112746,0.169990,0.288016,0.087908,0.288016
9,0.193668,-0.132523,0.085053,0.055732,-0.030748,0.039435,0.075579,-0.071394,0.044548,0.082377,...,0.015402,0.287428,0.052737,0.165457,0.261867,0.087307,0.169056,0.279438,0.051504,0.279438


In [39]:
for i in diff.index:
    diff.at[i,'mhtS2'] = sum(abs(j) for j in diff.iloc[i,0:45])
    diff.at[i,'eldS2'] = np.sqrt(sum(j**2 for j in diff.iloc[i,0:45]))

In [40]:
diff.head()

,FL/LT/Z5_zscore,FL/BL/Z5_zscore,FL/ST/Z5_zscore,AL/LT/Z5_zscore,AL/BL/Z5_zscore,AL/ST/Z5_zscore,AT/LT/Z5_zscore,AT/BL/Z5_zscore,AT/ST/Z5_zscore,FL/LT/Z3_zscore,...,FL/BLD/all_zscore,FL/STO/all_zscore,AL/LT/all_zscore,AL/BLD/all_zscore,AL/STO/all_zscore,AT/LT/all_zscore,AT/BLD/all_zscore,AT/STO/all_zscore,mhtS2,eldS2
0,0.220546,-0.117471,0.413964,0.062170,-0.305161,0.195495,0.087380,-0.430782,-0.274632,0.091404,...,0.648866,0.820346,0.269862,0.236487,0.635515,0.292087,0.381020,0.292087,9.842645,1.875302
1,-4.163675,-7.494650,-4.812008,-0.369725,-0.175860,0.528963,-1.276598,-3.389930,-2.180664,-1.716831,...,-2.939777,-1.891134,-0.724328,-1.115619,-0.417739,-1.241731,-2.847641,-1.241735,82.484453,16.517342
2,0.046199,0.068684,0.051182,-0.081513,0.010425,0.021140,0.010832,0.032518,0.026023,0.032851,...,0.148804,0.147901,0.091461,0.130558,0.148026,0.210040,0.145324,0.210040,2.469953,0.553972
3,0.182030,-0.032864,-0.087035,0.019079,-0.077140,-0.062674,0.070469,-0.026749,-0.073164,0.078468,...,0.102185,0.062348,0.216358,0.042428,0.057477,0.273961,0.094461,0.273961,3.216550,0.671328
4,-0.275280,-9.588497,-28.291116,-0.402502,-8.110054,-14.317611,-0.130316,-4.471276,-15.021824,-0.075115,...,-3.939489,-14.060928,-0.307087,-8.450392,-16.484097,0.058753,-3.823973,0.058753,250.601865,59.678113


In [58]:
s2=diff.iloc[:,-2:]

In [59]:
s2.head()

,mhtS2,eldS2
0,9.842645,1.875302
1,82.484453,16.517342
2,2.469953,0.553972
3,3.216550,0.671328
4,250.601865,59.678113


In [46]:
s2.to_csv("score2.csv")

In [61]:
s2=pd.read_csv('score2.csv')

In [62]:
s2.head()

,Unnamed: 0,mhtS2,eldS2
0,0,9.842645,1.875302
1,1,82.484453,16.517342
2,2,2.469953,0.553972
3,3,3.216550,0.671328
4,4,250.601865,59.678113


In [48]:
s1 = pd.read_csv('score1.csv')

In [57]:
s1.head()

,Unnamed: 0,mht,eld
0,0,2.162666,0.825242
1,1,22.385789,9.404851
2,2,0.642065,0.273179
3,3,0.773934,0.338544
4,4,72.370787,28.899709


In [64]:
import pandas as pd
ws=pd.merge(s1, s2,on="Unnamed: 0", how='outer')

In [65]:
ws.head()

,Unnamed: 0,mht,eld,mhtS2,eldS2
0,0,2.162666,0.825242,9.842645,1.875302
1,1,22.385789,9.404851,82.484453,16.517342
2,2,0.642065,0.273179,2.469953,0.553972
3,3,0.773934,0.338544,3.216550,0.671328
4,4,72.370787,28.899709,250.601865,59.678113


In [66]:
ws = ws.drop('Unnamed: 0',1)

In [67]:
ws.head()

,mht,eld,mhtS2,eldS2
0,2.162666,0.825242,9.842645,1.875302
1,22.385789,9.404851,82.484453,16.517342
2,0.642065,0.273179,2.469953,0.553972
3,0.773934,0.338544,3.216550,0.671328
4,72.370787,28.899709,250.601865,59.678113


In [71]:
ws['final']=0.8*ws['mht']+0.2*ws['mhtS2']

In [72]:
ws.head()

,mht,eld,mhtS2,eldS2,final
0,2.162666,0.825242,9.842645,1.875302,3.698662
1,22.385789,9.404851,82.484453,16.517342,34.405521
2,0.642065,0.273179,2.469953,0.553972,1.007642
3,0.773934,0.338544,3.216550,0.671328,1.262457
4,72.370787,28.899709,250.601865,59.678113,108.017003


In [76]:
ws.sort_values(by='final', ascending=False, na_position='first')

,mht,eld,mhtS2,eldS2,final
632815,2499.175043,1022.232337,16066.887695,3276.122940,5212.717573
565391,1821.355948,807.135118,10417.962578,2134.806164,3540.677274
1067359,1649.452235,821.811779,8750.338845,2273.339823,3069.629557
67128,1275.628920,627.048652,5054.072973,1160.975799,2031.317731
585117,1320.226676,616.176703,4304.025181,1122.877206,1916.986377
776305,1289.394522,535.530583,4248.801793,1311.834642,1881.275976
585438,1131.195974,520.309947,3542.371420,967.416253,1613.431063
918203,958.797597,384.093816,4184.254492,952.947393,1603.888976
85885,977.153320,395.747138,3764.131932,901.089053,1534.549043
920627,931.742096,424.015276,3115.140396,876.175128,1368.421756
